### Load the Commit Features

In [ ]:
project_root_path = '../../..'
lib_path = project_root_path + '/python/lib/'
import json
import sys
sys.path.append(lib_path)
import commit_features

In [3]:
import json
in_path = project_root_path + '/python/cfg/ds_pipeline.json'
config_map = json.loads(open(in_path, 'r').read())
config_map

cfg = config_map['99_problems']

In [4]:
project_stats_dir = project_root_path + '/' + cfg['stats_dir']
file_name = cfg['stat_file_name']
all_commit_features = commit_features.load_all(project_root_path, project_stats_dir, file_name)

In [6]:
X = commit_features.make_matrix(all_commit_features, cfg['classifier']['features'])

In [1]:
project_root_path = '../../..'
lib_path = project_root_path + '/python/lib/'
import json
import sys
sys.path.append(lib_path)
import commit_features

dataset_dir = project_root_path + '/sandbox/data/bob'
project_stats_dir = dataset_dir + '/ocean_repos'
file_name = 'commit_stat.json'
classifier_features = [
    'textFiles', 'textLines', 'configurationLines',
    'htmlLines', 'javascriptLines', 'jsonLines',
    'markdownLines', 'pythonLines',
    'solidityLines', 'yarnLines'
]

all_commit_features = commit_features.load_all(project_stats_dir, file_name)
X = commit_features.make_matrix(all_commit_features, classifier_features)

### Load the Trained Model

In [2]:
import pickle
model_path = project_root_path + '/sandbox/data/bob/logit-clf.dat'
fin = open(model_path, 'rb')
model_deserial = pickle.load(fin)
fin.close()
y_pred = model_deserial.predict(X)
print(len(y_pred))

93960


### Compile Hacker Histories

In [3]:
hacker_histories = {}
for i in range(len(y_pred)):
    features = all_commit_features[i]
    pred = y_pred[i]
    author = features['author']
    features['predicted_class'] = pred
    if author not in hacker_histories: hacker_histories[author] = []
    hacker_commits = hacker_histories[author]
    hacker_commits.append(features)

### Show Hacker Commit Types

In [8]:
import matplotlib.pyplot as plt

hacker_histograms = []
for hacker in hacker_histories.keys():
    commits = hacker_histories[hacker]
    commit_types = [commit['predicted_class'] for commit in commits]
    observed_types = list(set(commit_types))
    hacker_commit_types = []
    for observed_type in observed_types:
        observations = [1 for commit_type in commit_types if commit_type == observed_type]
        count = len(observations)
        hacker_commit_types.append(dict(num_commits=count, commit_type=observed_type))
    sorted_list = sorted(hacker_commit_types, key=lambda d: d['num_commits'])
    sorted_list.reverse()
    hacker_histograms.append(dict(hacker=hacker, commit_stats=sorted_list))

sorted_histograms = sorted(hacker_histograms, key=lambda d: d['hacker'].lower())
for histo in sorted_histograms:
    if histo['commit_stats'][0]['num_commits'] > 2000:
        print(histo['hacker'])
        for row in histo['commit_stats']:
            print(str(row['num_commits']) + "\t" + str(row['commit_type']))
        print()
        #break

alexcos20 <alex.coseru@gmail.com>
3006	js
2810	minor_js
2394	solidity
952	minor_py
939	bulk
756	md
738	json
679	python
453	minor_yarn
306	big_js
216	minor_json
90	markdown
54	yarn
54	minor_md
54	big_solidity
38	big_py
36	config
18	minor

Bogdan Fazakas <bogdan.fazakas@gmail.com>
2214	js
774	minor_js
504	bulk
486	big_js
90	interface
72	json
36	big_json
18	minor_yarn
18	markdown
18	minor_html

lacoop6tu <burzi.andrea@gmail.com>
2772	bulk
1440	solidity
774	js
666	big_js
450	minor_js
126	big_solidity
90	markdown
72	minor_md

Matthias Kretschmann <m@kretschmann.io>
7902	js
4086	minor_js
3546	interface
1638	bulk
1170	json
954	md
486	big_js
468	big_json
252	markdown
236	minor_md
235	minor_yarn
180	minor_html
109	minor_py
54	yarn
38	python
36	minor
18	big_markdown

ssallam <travis@travis-ci.org>
2954	minor_py
2861	python
780	bulk
398	minor_yarn
364	minor_js
291	big_py
162	js
145	markdown
144	py
91	minor_md
73	config
56	minor
18	big_js
18	json

Trent McConaghy <5305452+trentmc@users.noreply.git

In [5]:
import json

out_path = project_root_path + '/sandbox/data/bob/hacker_history_2.json'
out_handle = open(out_path, "w");
out_handle.write(json.dumps(sorted_histograms, indent=2));
out_handle.flush()
out_handle.close()